# Pasos a seguir

## Pasos para realizar el reto de extracción de información de facturas eléctricas

**1. Recopilación de recursos:**

* Aseguro tener instalado Python en mi ordenador.
* Descargo el dataset de entrenamiento que contiene las facturas PDF y sus correspondientes archivos JSON.
* Investigo y selecciono las librerías de Python adecuadas para:
    * Extraer texto de archivos PDF: Considero librerías como `PyPDF2`, `Poppler` o `pdfminer.six`.
    * Procesamiento de texto: Utilizo librerías como `re`, `string` o `nltk` para la limpieza y manipulación del texto extraído.
    * Aprendizaje automático: Si decido utilizar un modelo de aprendizaje automático para la extracción de información, necesitaré librerías como `scikit-learn` o `TensorFlow`.

**2. Preprocesamiento de datos:**

* Leo los archivos PDF y JSON del dataset de entrenamiento.
* Limpio el texto extraído de los PDF, eliminando caracteres irrelevantes, convirtiendo todo a minúsculas y manejando correctamente los caracteres con tilde.
* Estructuro los datos de manera que se facilite su procesamiento, por ejemplo, creando un diccionario para cada factura que contenga los campos extraídos del PDF y su correspondiente valor del archivo JSON.

**3. Exploración y análisis de datos:**

* Analizo la estructura y el contenido de las facturas PDF y los archivos JSON para identificar patrones y características comunes.
* Identifico posibles variaciones en el formato y la disposición de la información dentro de las facturas.
* Segmento los datos en diferentes grupos según sus características, por ejemplo, por tipo de comercializadora o formato de factura.

**4. Definición de la estrategia de extracción:**

* Decido si utilizar un enfoque basado en reglas o un modelo de aprendizaje automático para la extracción de información.
* **Enfoque basado en reglas:**
    * Defino reglas manuales para identificar y extraer cada campo de información en base a su ubicación, formato y características dentro del PDF.
    * Considero el uso de expresiones regulares o técnicas de procesamiento del lenguaje natural para la extracción precisa de la información.
* **Enfoque basado en aprendizaje automático:**
    * Elijo un algoritmo de aprendizaje automático adecuado para la tarea de extracción de información, como clasificación o aprendizaje supervisado.
    * Entreno el modelo utilizando los datos preprocesados del dataset de entrenamiento.
    * Evalúo el rendimiento del modelo en un conjunto de datos de validación para asegurar su generalización a nuevas facturas.

**5. Implementación de la solución:**

* Implemento el algoritmo de extracción de información, ya sea basado en reglas o en aprendizaje automático.
* Desarrollo la lógica para procesar cada factura PDF, extraer la información relevante y almacenarla en un formato estructurado.
* Manejo posibles casos de error o situaciones donde la información no se encuentre en el formato esperado.

**6. Evaluación y mejora:**

* Evalúo el rendimiento de la solución utilizando el script proporcionado para obtener el score de Levenshtein.
* Analizo los resultados obtenidos para identificar áreas de mejora y posibles errores en la extracción de información.
* Refino la estrategia de extracción, las reglas o el modelo de aprendizaje automático en base a los resultados de la evaluación.
* Repito el proceso de evaluación y mejora hasta obtener un score satisfactorio.

**7. Consideraciones adicionales:**

* Es importante tener en cuenta la diversidad de formatos y estructuras de las facturas para garantizar la generalización del método de extracción.
* La utilización de técnicas de procesamiento del lenguaje natural puede ser útil para mejorar la precisión de la extracción, especialmente en casos de ambigüedad o falta de información.
* Es recomendable implementar mecanismos para manejar errores y casos excepcionales, como la ausencia de campos o formatos de datos inesperados.
* La documentación del código y la estrategia de extracción es fundamental para facilitar su comprensión y mantenimiento.

# Librerías

Para realizar la limpieza del texto extraído de los PDFs, te recomiendo utilizar las siguientes librerías de Python:

**re:** Módulo para expresiones regulares, que te permitirá identificar y eliminar patrones de caracteres irrelevantes.

**string:** Módulo que contiene funciones útiles para el manejo de cadenas de texto, como la conversión a minúsculas.

**unicodedata:** Módulo que proporciona funciones para trabajar con caracteres Unicode, incluyendo la normalización de caracteres con tilde.

In [250]:
import pandas as pd
import numpy as np
from datetime import datetime

# Leer pdf
import PyPDF2
#import pdfplumber
import fitz  # PyMuPDF
#from pdfminer.high_level import extract_text
import PyPDF4
import slate3k as slate

# Procesamiento texto
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import unicodedata

# Guardar archivos
import pickle

# Base de datos sql
import mysql.connector

# Buscar provincia a través de código postal
import pgeocode

# Datos a extraer:

- Nombre del cliente (nombre_cliente) 
- DNI del cliente (dni_cliente) 
- Calle del clilente (calle_cliente) 
- Código postal del cliente (cp_cliente) 
- Población del cliente (población_cliente) 
- Provincia del cliente (provincia_cliente) 
- Nombre de la empresa comercializadora (nombre_comercializadora)
- CIF de la comercializadora (cif_comercializadora) 
- Dirección de la comercializadora (dirección_comercializadora)
- Código postal de la comercializadora (cp_comercializadora)
- Población de la comercializadora (población_comercializadora)
- Provincia de la comercializadora (provincia_comercializadora)
- Número de factura (nümero_factura)
- Inicio del periodo de facturación (inicio_periodo)
- Fin del periodo de facturacón (fin_periodo)
- Importe de la factura (importe_factura)
- Fecha del cargo (fecha_cargo)
- Consumo en el periodo (consumo_periodo)
- Potencia contratada (potencia_contratada)

### **Leer los archivos**

In [ ]:
# Abre el archivo PDF en modo lectura binaria
invoices = {}
errors = {}
count = 0

for i in range(1000):
    try:
        with open(f'./training/factura_{i}.pdf', 'rb') as file:
            print(f'./training/factura_{i}.pdf')
            reader = PyPDF2.PdfReader(file)
            # Obtén el número de páginas
            num_pages = len(reader.pages)
            content = {}
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()
                content[page_num] = text
            invoices[i] = content
    except:
        print("Error al leer la factura")
        errors[count] = f"factura_{i}"
        count += 1
    print(f"{i}")

In [ ]:
# Guardar el diccionario en un archivo
with open('facturas.pkl', 'wb') as file:
    pickle.dump(invoices, file)

In [64]:
# Función para leer una factura:
#name = "nombre archivo sin extension"

def ft_readinvoice(path, name):
    invoices = {}
    errors = {}
    count = 0
    try:
        with open(path, 'rb') as file:
            print(name)
            reader = PyPDF2.PdfReader(file)
            # Obtén el número de páginas
            num_pages = len(reader.pages)
            content = {}
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()
                content[page_num] = text
            invoices[0] = content
        with open(f"{name}.pkl", 'wb') as file:
            pickle.dump(invoices, file)
    except:
        print("""Error al leer la factura. 
            Comprueba que el archivo esté en el mismo directorio, 
            que el nombre del archivo sea válido (sin la extensión) 
            y que el archivo sea formato pdf""")
        errors[count] = name
        count += 1
        
    return invoices

In [65]:
path = "./training/factura_124.pdf"
name = "Factura_124"
factura = ft_readinvoice(path, name)

Factura_124
Factura_124.pkl


In [66]:
factura

{0: {0: "        L...u..n..e..s.. .a.. .s..á..b..a..d..o..,. .d..e.. .8.. .a.. .2..2.. .h..o..r.a..s.......................... \nContratación Productos y Servicios  XXXXXXX  \nReclamaciones  e incidencias    023  \n \nBAHNHOFQUAI, 12, 4600 - Olten (Suiza)  \nSi quiere  una atención  más personalizada  puede  acudir  a alguno  de los centros  que ALPIQ TRADING AG tiene  a su  \nservicio. Encuentre el más cercano en /centros o desde su móvil en  /.mobi.  2..4.. .h..o...r.a..s.. ./. .3..6..5.. .d..í.a..s.. .d..e..l. .a..ñ..o................................... \nWeb     \nReparación  urgente  de calderas  \ny electrodomésticos  XXX XXX XXX  \nLectura  del contador  XXX  XXX XXX  \n/lecturas  \nAverías  eléctricas  061   \n \n \n  ALPIQ TRADING AG  \n Fecha  de emisión:  10.09.2011   \nNº factura:  T9396049062  \nNombre:  ROBINSON SÁNCHEZ VELA  \nDirección  suministro:  Calle del Arquitecto Sánchez Arcas  \n09217 Condado de Treviño  \nNº Referencia:  650856284802/3925  \nNº cliente:  XXXXXX

# Procesamiento de datos

### Funciones necesarias

* **Eliminar caracteres irrelevantes:** Utiliza expresiones regulares para eliminar caracteres como tabuladores, retornos de carro, caracteres de control y cualquier otro símbolo que no sea relevante para la extracción de información.
* **Convertir a minúsculas:** Convierte todo el texto a minúsculas utilizando la función `lower()` del módulo `string`.
* **Manejar caracteres con tilde:** Normaliza los caracteres con tilde utilizando la función `normalize()` del módulo `unicodedata` con el argumento `'NFKD'`. Esto descompone los caracteres con tilde en dos caracteres: la letra base y el acento.

In [55]:
# Función normalización de factura
def ft_cleaner(text):
    # Eliminar saltos de línea y tabuladores
    text = re.sub(r'[\n\t]', ' ', text)
    
    # Normalizar el texto (NFKD) y eliminar diacríticos (acentos)
    normalize_text = unicodedata.normalize('NFKD', text)
    text_tilde = ''.join(c for c in normalize_text if not unicodedata.combining(c))
    
    # Eliminar caracteres especiales adicionales, dejando solo letras, dígitos, espacios, guiones, comillas simples y comas
    final_text = re.sub(r"[^A-Za-z0-9\s\',/.]", '', text_tilde)
    
    # Quitar espacios adicionales
    final_text = ' '.join(final_text.split())

    final_text = final_text.lower()
    
    return final_text

In [29]:
with open('facturas.pkl', 'rb') as file:
    invoices = pickle.load(file)

clean_invoices = {}

for tittle, pages in invoices.items():
    clean_pages = {}
    for num_pages, content in pages.items():
        clean_pages[num_pages] = ft_cleaner(content)
        #clean_pages[num_pages] = unicodedata.normalize('NFKD', content)
    clean_invoices[tittle] = clean_pages

# Guardar el diccionario en un archivo
with open('facturas_limpias.pkl', 'wb') as file:
    pickle.dump(clean_invoices, file)

In [85]:
# Función para obtener factura normalizada
def ft_normalize(name):
    print(name)
    
    pickle_file = f"{name}.pkl"
    with open(pickle_file, 'rb') as file:
        invoices = pickle.load(file)
        
    clean_invoices = {}

    for tittle, pages in invoices.items():
        clean_pages = {}
        for num_pages, content in pages.items():
            clean_pages[num_pages] = ft_cleaner(content)
            #clean_pages[num_pages] = unicodedata.normalize('NFKD', content)
        clean_invoices[tittle] = clean_pages
    clean_invoices = clean_invoices[0]

    # Guardar el diccionario en un archivo
    with open(f'{name}_limpia.pkl', 'wb') as file:
        pickle.dump(clean_invoices, file)
    
    return clean_invoices

In [86]:
path = "./training/factura_124.pdf"
name = "Factura_124"
factura = ft_readinvoice(path, name)
factura_limpia = ft_normalize(name)

Factura_124


In [87]:
factura_limpia

{0: "l...u..n..e..s.. .a.. .s..a..b..a..d..o..,. .d..e.. .8.. .a.. .2..2.. .h..o..r.a..s.......................... contratacion productos y servicios xxxxxxx reclamaciones e incidencias 023 bahnhofquai, 12, 4600 olten suiza si quiere una atencion mas personalizada puede acudir a alguno de los centros que alpiq trading ag tiene a su servicio. encuentre el mas cercano en /centros o desde su movil en /.mobi. 2..4.. .h..o...r.a..s.. ./. .3..6..5.. .d..i.a..s.. .d..e..l. .a..n..o................................... web reparacion urgente de calderas y electrodomesticos xxx xxx xxx lectura del contador xxx xxx xxx /lecturas averias electricas 061 alpiq trading ag fecha de emision 10.09.2011 no factura t9396049062 nombre robinson sanchez vela direccion suministro calle del arquitecto sanchez arcas 09217 condado de trevino no referencia 650856284802/3925 no cliente xxxxxxx nif 45498212a direccion calle del arquitecto sanchez arcas 09217 condado de trevino entidad novo banco, s.a., sucursal en e

# Extracción:

- Nombre del cliente (nombre_cliente) **HECHO**
- DNI del cliente (dni_cliente) **HECHO**
- Calle del clilente (calle_cliente) **HECHO**
- Código postal del cliente (cp_cliente) **HECHO**
- Población del cliente (población_cliente) **HECHO**
- Provincia del cliente (provincia_cliente) **HECHO**
- Nombre de la empresa comercializadora (nombre_comercializadora) **HECHO MENOS 46**
- CIF de la comercializadora (cif_comercializadora) **HECHO MENOS FACTURA 631**
- Dirección de la comercializadora (dirección_comercializadora) **HECHO MENOS 32**
- Código postal de la comercializadora (cp_comercializadora) **HECHO MENOS 48**
- Población de la comercializadora (población_comercializadora) **HECHO MENOS 30**
- Provincia de la comercializadora (provincia_comercializadora) **HECHO**
- Número de factura (nümero_factura)**HECHO**
- Inicio del periodo de facturación (inicio_periodo) **HECHO**
- Fin del periodo de facturacón (fin_periodo) **HECHO**
- Importe de la factura (importe_factura) **HECHO**
- Fecha del cargo (fecha_cargo) **HECHO**
- Consumo en el periodo (consumo_periodo) **HECHO**
- Potencia contratada (potencia_contratada) **HECHO**

In [39]:
with open('facturas_limpias.pkl', 'rb') as file:
    clean_invoices = pickle.load(file)

In [70]:
def encontrar_provincia_por_cp(codigo_postal, pais='ES'):
    # Crear un objeto Nominatim para el país especificado (por defecto España)
    nomi = pgeocode.Nominatim(pais)
    
    # Obtener la información geográfica del código postal
    info = nomi.query_postal_code(codigo_postal)
    
    # Verificar si se encontró información válida
    if info.county_name:
        return info.county_name
    else:
        return "none"

In [308]:
def ft_font_code(invoice, name):

    result = {}

    invoices = {}
    client_name = []
    dni = []
    adress_street = []
    adress_city = []
    adress_province = []
    adress_cp = []
    distributor_name = []
    cif = []
    distributor_city = []
    distributor_street = []
    distributor_province = []
    distributor_cp = []

    pages = invoice

    for n_pages, content in pages.items():
    
        # -----------------------------------------------------------------------------------------------------------
        # Client name
        patron_cn_1 = r'nombre (.*?) direccion'
        matches_1 = re.findall(patron_cn_1, content, re.IGNORECASE)
        for match in matches_1:
            if match:
                client_name.extend(matches_1)
        
        patron_cn_2 = r'titular del contrato (.*?) nif'
        matches_2 = re.findall(patron_cn_2, content, re.IGNORECASE)
        for match in matches_2:
            if match not in client_name:  # Solo agregar si no está en matches_1
                client_name.append(match)
        patron_cn_3 = r'titular(?! del contrato)(.*?)nif'
        matches_3 = re.findall(patron_cn_3, content, re.IGNORECASE)
        for match in matches_3:
            if match not in client_name:  # Solo agregar si no está en matches_1
                client_name.append(match)
        invoices["nombre_cliente"] = client_name[:]
        
        # ----------------------------------------------------------------------------------------------------------------------------
        # DNI   
        patron_nif = r'nif (\d{8}[a-zA-Z])'
        nif_1 = re.findall(patron_nif, content, re.IGNORECASE)
        for match in nif_1:
            if match not in dni:
                dni.extend(nif_1)
        invoices["dni_cliente"] = dni[:]

        #-----------------------------------------------------------------------------------------------------------------------------
        # Adress, city, province, cp
        patron_adress_1 = r"direccion\sde\ssuministro\s(?P<calle>.+?),\s(?P<localidad>.+?),\s(?P<provincia>.+?)\snumero\sde\scontador"
        adress_1 = re.search(patron_adress_1, content)
        if adress_1:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_1.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "población_cliente" not in invoices:
                adress_city.append(adress_1.group("localidad"))
                invoices["población_cliente"] = adress_city
            if "provincia_cliente" not in invoices:
                adress_province.append(adress_1.group("provincia"))
                invoices["provincia_cliente"] = adress_province

            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_1.group("calle")
                found_city = adress_1.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, pages[n_pages - 1])

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress_2 = r"direccion\s+suministro\s+(?P<calle>.+?)\s+(?P<codigo_postal>\d{5})\s+(?P<localidad>.+?)\b"
        adress_2 = re.search(patron_adress_2, content)
        if adress_2:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_2.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_2.group("codigo_postal"))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_2.group("localidad"))
                invoices["población_cliente"] = adress_city
            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_2.group("calle")
                found_city = adress_2.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, content)

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress = r"direccion de suministro\s+(?P<calle>.+?),\s+(?P<poblacion>.+?),\s+(?P<provincia>\w+)\s+numero de contador"
        adress_match = re.search(patron_adress, content)

        if adress_match:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_match.group("calle"))
                invoices["calle_cliente"] = adress_street

            poblacion = adress_match.group("poblacion")
            if "población_cliente" not in invoices:
                invoices["población_cliente"] = [poblacion]
            elif len(poblacion) < len(invoices["población_cliente"][0]):
                invoices["población_cliente"] = [poblacion]

            if "provincia_cliente" not in invoices:
                adress_province.append(adress_match.group("provincia"))
                invoices["provincia_cliente"] = adress_province

        patron_direccion = r"calle\s+(?P<calle>.+?)\s+(?P<cp>\d{5})\s+(?P<localidad>.+?)\s+(?P<provincia>\w+)$"
        direccion_match = re.match(patron_direccion, content)

        if direccion_match:
            if "calle_cliente" not in invoices:
                adress_street.append(direccion_match.group("calle"))
                invoices["calle_cliente"] = adress_street

            poblacion = direccion_match.group("localidad")
            if "población_cliente" not in invoices:
                invoices["población_cliente"] = [poblacion]
                
            elif len(poblacion) < len(invoices["población_cliente"][0]):
                invoices["población_cliente"] = [poblacion]

            if "provincia_cliente" not in invoices:
                adress_province.append(direccion_match.group("provincia"))
                invoices["provincia_cliente"] = adress_province

        patron_adress_3 = r"calle\s+(?P<calle>.+?)\s+(\d{5})\s+(?P<localidad>.+?)\s+forma\s+de\s+pago\s+domiciliada"
        adress_3 = re.search(patron_adress_3, content)
        if adress_3:
            if "calle_cliente" not in invoices:
                adress_street.append("calle" + adress_3.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_3.group(2))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_3.group("localidad"))
                invoices["población_cliente"] = adress_city
            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_3.group("calle")
                found_city = adress_3.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, content)

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress_4 = r"direccion\sde\ssuministro\s(?P<calle>.+?)\s+(?P<codigo_postal>\d{5})\s+(?P<localidad>\w+)"
        adress_4 = re.search(patron_adress_4, content)
        if adress_4:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_4.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_4.group("codigo_postal"))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_4.group("localidad"))
                invoices["población_cliente"] = adress_city

            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_4.group("calle")
                found_city = adress_4.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, pages[n_pages - 1])

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp[0]
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
        
        #----------------------------------------------------------------------------------------------
        # CIF de la empresa comercializadora
        patron_cif_1 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\."
        cif_match = re.search(patron_cif_1, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif
        # CIF de la empresa comercializadora
        patron_cif_2 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\s*\."
        cif_match = re.search(patron_cif_2, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif
        
        # CIF de la empresa comercializadora
        patron_cif_3 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\b"
        cif_match = re.search(patron_cif_3, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_4 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,8})\s*\."
        cif_match = re.search(patron_cif_4, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_5 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,8})\b"
        cif_match = re.search(patron_cif_5, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        patron_cif_6 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,9})\b"
        cif_match = re.search(patron_cif_6, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_1 = r"cif\s+(?P<cif>[a-zA-Z]{1,3}\s?\d{3}(\.?\d{3}){1,2}|\d{3}'\d{3}|\d{9,10})[\s\.]*"
        cif_match = re.search(patron_cif_1, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        #-----------------------------------------------------------------------------------------------
        # Dirección de la empresa 
        if (invoices['nombre_cliente']):
            if ("cif_comercializadora" in invoices):
                patron_location_1 = r"(?P<nombre_empresa>[^\d.]+)\s*\. cif\s*{}\s*\.\s*(?P<direccion>.*?)\s*(?P<cp>\d{{5}})\s*(?P<localidad>[^\s]*).*{}".format(invoices["cif_comercializadora"][0], re.escape(invoices["nombre_cliente"][0]))
                if (n_pages >= 1):
                    location_match = re.search(patron_location_1, pages[n_pages - 1], re.IGNORECASE)
                else:
                    location_match = re.search(patron_location_1, pages[n_pages], re.IGNORECASE)

                if location_match:
                
                    if "nombre_comercializadora" not in invoices:
                        distributor_name.append(location_match.group('nombre_empresa'))
                        invoices["nombre_comercializadora"] = distributor_name
    
                    if "dirección_comercializadora" not in invoices:
                        distributor_street.append(location_match.group('direccion').strip())
                        invoices["dirección_comercializadora"] = distributor_street
    
                    if "cp_comercializadora" not in invoices:
                        distributor_cp.append(location_match.group('cp'))
                        invoices["cp_comercializadora"] = distributor_cp
    
                    if "localidad_comercializadora" not in invoices:
                        distributor_city.append(location_match.group('localidad'))
                        invoices["localidad_comercializadora"] = distributor_city

                patron_location_2 = r"fecha de cargo.*?(?P<nombre_empresa>[^\d]+?)\s+cif\s+{}\s+(?P<direccion>.*?)\s+(?P<cp>\d{{5}})\s+(?P<localidad>[^\s]+)\s+{}".format(invoices["cif_comercializadora"][0], re.escape(invoices["nombre_cliente"][0]))
                if (n_pages >= 1):
                    location_match = re.search(patron_location_2, pages[n_pages - 1], re.IGNORECASE)
                else:
                    location_match = re.search(patron_location_2, pages[n_pages], re.IGNORECASE)

                if location_match:
                
                    if "nombre_comercializadora" not in invoices:
                        distributor_name.append(location_match.group('nombre_empresa'))
                        invoices["nombre_comercializadora"] = distributor_name
    
                    if "dirección_comercializadora" not in invoices:
                        distributor_street.append(location_match.group('direccion').strip())
                        invoices["dirección_comercializadora"] = distributor_street
    
                    if "cp_comercializadora" not in invoices:
                        distributor_cp.append(location_match.group('cp'))
                        invoices["cp_comercializadora"] = distributor_cp
    
                    if "localidad_comercializadora" not in invoices:
                        distributor_city.append(location_match.group('localidad'))
                        invoices["localidad_comercializadora"] = distributor_city

        patron_location_3 = r"(?<=mifactura\s)(?P<nombre_empresa>[A-Za-z\s,\.]+?)\s*\.?\s*dom\. social\s+(?P<direccion>.*?),\s*(?P<cp>\d{5})\s+(?P<localidad>[^\.\,]+)\.\s*(?P<provincia>[^\.\,]+)"
        location_match = re.search(patron_location_3, pages[n_pages], re.IGNORECASE)

        if location_match:
        
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(location_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name

            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street

            if "cp_comercializadora" not in invoices:
                distributor_cp.append(location_match.group('cp'))
                invoices["cp_comercializadora"] = distributor_cp

            if "localidad_comercializadora" not in invoices:
                distributor_city.append(location_match.group('localidad'))
                invoices["localidad_comercializadora"] = distributor_city

            if "provincia_comercializadora" not in invoices:
                distributor_province.append(location_match.group('provincia'))
                invoices["provincia_comercializadora"] = distributor_province

        if ("cif_comercializadora" in invoices):
            patron_location_4 = r"(?P<nombre_empresa>[^\d]+?)\s+cif\s+{}\s+(?P<direccion>.*?)\s+(?P<cp>\d{{5}})\s+(?P<localidad>[^\s]+)".format(re.escape(invoices["cif_comercializadora"][0]))
            if (n_pages >= 1):
                location_match = re.search(patron_location_4, pages[n_pages - 1], re.IGNORECASE)
            else:
                location_match = re.search(patron_location_4, pages[n_pages], re.IGNORECASE)

            if location_match:
            
                if "nombre_comercializadora" not in invoices:
                    distributor_name.append(location_match.group('nombre_empresa'))
                    invoices["nombre_comercializadora"] = distributor_name

                if "dirección_comercializadora" not in invoices:
                    distributor_street.append(location_match.group('direccion').strip())
                    invoices["dirección_comercializadora"] = distributor_street

                if "cp_comercializadora" not in invoices:
                    distributor_cp.append(location_match.group('cp'))
                    invoices["cp_comercializadora"] = distributor_cp

                if "localidad_comercializadora" not in invoices:
                    distributor_city.append(location_match.group('localidad'))
                    invoices["localidad_comercializadora"] = distributor_city

        patron_location_5 =  r"\/mifactura\s+(?P<nombre_empresa>.*?)\.\s+dom\.?\s+social\s+(?P<direccion>.*?)\,\s+(?P<cp>\d{4})\s+(?P<localidad>[\w\s]+)\.\s+(?P<provincia>[\w\s]+),"
        location_match = re.search(patron_location_5, pages[n_pages], re.IGNORECASE)

        if location_match:
        
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(location_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name

            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street

            if "provincia_comercializadora" not in invoices:
                distributor_province.append(location_match.group('provincia'))
                invoices["provincia_comercializadora"] = distributor_province
        
        patron_location_6 = r"domicilio\s+social\s+(?P<direccion>[^,]+),\s*(?P<localidad>[^\.,]+) xxxxx"
        location_match = re.search(patron_location_6, pages[n_pages])

        if location_match:
            
            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street
            if "localidad_comercializadora" not in invoices:
                distributor_city.append(location_match.group('localidad').strip())
                invoices["localidad_comercializadora"] = distributor_city

        # --------------------------------------------------------------------------------------------
        # Nombre de la comercializadora
        patron_name_1 = r"[\s.]+(?P<nombre_empresa>[^\.,]+)\.\s+inscrita"
        name_match = re.search(patron_name_1, pages[n_pages])

        if name_match:
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(name_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name

        # ----------------------------------------------------------------------------------------------
        # Patrón para el importe de la factura
        importe_pattern = r"importe factura (\d{1,3}(?:\.\d{3})*(?:,\d{2})?)"
        # Patrón para el número de factura
        invoice_number_pattern = r"no factura (\w+)"
        # Patrón para las fechas de inicio y final de facturación
        dates_pattern = r"periodo de consumo (\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})"
        # Patrón para la fecha de cargo
        cargo_date_pattern = r"fecha de cargo (\d{1,2} de \w+ de \d{4})"

         # Inicializar las variables
        importe = None
        invoice_number = None
        start_date = None
        end_date = None
        cargo_date = None

        # Buscar los patrones en el texto
        importe_match = re.search(importe_pattern, pages[n_pages])
        if importe_match:
            if ("importe_factura" not in invoices):
                importe = importe_match.group(1)
                invoices["importe_factura"] = importe
            
        invoice_number_match = re.search(invoice_number_pattern, pages[n_pages])
        if invoice_number_match:
            if ("número_factura" not in invoices):
                invoice_number = invoice_number_match.group(1)
                invoices["número_factura"] = invoice_number
        
        dates_match = re.search(dates_pattern, pages[n_pages])
        if dates_match:
            if ("inicio_periodo" not in invoices):
                start_date = dates_match.group(1)
                invoices["inicio_periodo"] = start_date
            if ("fin_periodo" not in invoices):
                end_date = dates_match.group(2)
                invoices["fin_periodo"] = end_date

        dates_pattern = r"del (\d{2}\.\d{2}\.\d{4}) al (\d{2}\.\d{2}\.\d{4})"
        # Buscar el patrón en el texto
        dates_match = re.search(dates_pattern, pages[n_pages])
        if dates_match:
            if ("inicio_periodo" not in invoices):
                start_date = dates_match.group(1)
                invoices["inicio_periodo"] = start_date
            if ("fin_periodo" not in invoices):
                end_date = dates_match.group(2)
                invoices["fin_periodo"] = end_date
        
        dates_pattern = r"periodo de facturacion del (\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})"
        dates_match = re.search(dates_pattern, pages[n_pages])
        if dates_match:
            if ("inicio_periodo" not in invoices):
                start_date = dates_match.group(1)
                invoices["inicio_periodo"] = start_date
            if ("fin_periodo" not in invoices):
                end_date = dates_match.group(2)
                invoices["fin_periodo"] = end_date
        
        dates_pattern = r"periodo de consumo (\d{1,2} de \w+ de \d{4}) a (\d{1,2} de \w+ de \d{4})"
        dates_match = re.search(dates_pattern, pages[n_pages])
        if dates_match:
            if ("inicio_periodo" not in invoices):
                start_date = dates_match.group(1)
                invoices["inicio_periodo"] = start_date
            if ("fin_periodo" not in invoices):
                end_date = dates_match.group(2)
                invoices["fin_periodo"] = end_date
        
        cargo_date_match = re.search(cargo_date_pattern, pages[n_pages])
        if cargo_date_match:
            if ("fecha_cargo" not in invoices):
                cargo_date = cargo_date_match.group(1)
                invoices["fecha_cargo"] = cargo_date
        
        cargo_date_pattern = r"fecha de cargo (\d{2}\.\d{2}\.\d{4})"
        cargo_date_match = re.search(cargo_date_pattern, pages[n_pages])
        if cargo_date_match:
            if ("fecha_cargo" not in invoices):
                cargo_date = cargo_date_match.group(1)
                invoices["fecha_cargo"] = cargo_date

        amount_pattern = r"importe de su factura, (\d+(?:,\d+)*(?:\.\d{2})?) euros"
        # Inicializar la variable
        invoice_amount = None
        # Buscar el patrón en el texto
        amount_match = re.search(amount_pattern, pages[n_pages])
        if amount_match:
            if ("importe_factura" not in invoices):
                invoice_amount = amount_match.group(1)
                invoices["importe_factura"] = invoice_amount

        # Patrón para el periodo de facturación seguido del consumo en kWh
        consumption_pattern = r"periodo de \d{2}\.\d{2}\.\d{4} a \d{2}\.\d{2}\.\d{4} (\d+) kwh \d+ dias"
        # Inicializar la variable
        consumption = None
        # Buscar el patrón en el texto
        consumption_match = re.search(consumption_pattern, pages[n_pages])
        if consumption_match:
            if ("consumo_periodo" not in invoices):
                consumption = consumption_match.group(1)
                invoices["consumo_periodo"] = consumption
        
        consumption_pattern = r"periodo de \d{2}\.\d{2}\.\d{4} a \d{2}\.\d{2}\.\d{4} (\d+) kwh"
        # Buscar el patrón en el texto
        consumption_match = re.search(consumption_pattern, pages[n_pages])
        if consumption_match:
            if ("consumo_periodo" not in invoices):
                consumption = consumption_match.group(1)
                invoices["consumo_periodo"] = consumption

        consumption_pattern = r"importe por energia consumida (\d+) kwh"
        # Buscar el patrón en el texto
        consumption_match = re.search(consumption_pattern, pages[n_pages])
        if consumption_match:
            consumption = int(consumption_match.group(1))
            if ("consumo_periodo" not in invoices):
                invoices["consumo_periodo"] = consumption

        consumption_pattern = r"consumo en el periodo (\d+) kwh"
        # Buscar el patrón en el texto
        consumption_match = re.search(consumption_pattern, pages[n_pages])
        if consumption_match:
            consumption = int(consumption_match.group(1))
            if ("consumo_periodo" not in invoices):
                invoices["consumo_periodo"] = consumption

        consumption_pattern = r"importe por peaje de acceso (\d+) kwh"
        # Buscar el patrón en el texto
        consumption_match = re.search(consumption_pattern, pages[n_pages])
        if consumption_match:
            consumption = int(consumption_match.group(1))
            if ("consumo_periodo" not in invoices):
                invoices["consumo_periodo"] = consumption

        power_pattern = r"potencia contratada ([\d,]+) kw"
        # Inicializar la variable
        power = None
        # Buscar el patrón en el texto
        power_match = re.search(power_pattern, pages[n_pages])
        if power_match:
            if "potencia_contratada" not in invoices:
                # Reemplazar la coma por un punto para asegurar el formato float
                power = power_match.group(1)
                invoices["potencia_contratada"] = power
          
    if (len(adress_province) == 0):
        adress_province = [encontrar_provincia_por_cp(invoices["cp_cliente"][0], "ES")]
        invoices["provincia_cliente"] = adress_province

    if (len(distributor_province) == 0):
        if ("cp_comercializadora" in invoices):
            adress_province = [encontrar_provincia_por_cp(invoices["cp_comercializadora"][0], "ES")]
            invoices["provincia_comercializadora"] = adress_province

    # Fill result
    result[name] = invoices

    
    return result

In [ ]:
# Código para todas las facturas

data_train = {}
for key, value in clean_invoices.items():
    data_train[f"Factura_{key}"] = {}
    result = ft_font_code(value, f"Factura_{key}")
    data_train[f"Factura_{key}"][f"Factura_{key}"] = result[f"Factura_{key}"]
    print(key)

In [306]:
path = "./training/factura_124.pdf"
name = "Factura_124"
factura = ft_readinvoice(path, name)
factura_limpia = ft_normalize(name)
result = ft_font_code(factura_limpia, name)

In [276]:
def ft_date_convert(date):
    meses = {
        'enero': 'January',
        'febrero': 'February',
        'marzo': 'March',
        'abril': 'April',
        'mayo': 'May',
        'junio': 'June',
        'julio': 'July',
        'agosto': 'August',
        'septiembre': 'September',
        'octubre': 'October',
        'noviembre': 'November',
        'diciembre': 'December'
    }
    try:
        # Convertir la cadena de fecha a minúsculas para manejar diferentes casos
        date = date.lower()
        
        # Traducir los nombres de los meses a inglés
        for mes_es, mes_en in meses.items():
            date = date.replace(mes_es, mes_en)
        
        # Probar diferentes formatos de fecha
        if 'de' in date:
            date = datetime.strptime(date, '%d %B %Y')
        elif '.' in date:
            date = datetime.strptime(date, '%d.%m.%Y')
        else:
            date = datetime.strptime(date, '%d/%m/%Y')
            
        date = date.strftime("%m.%d.%Y")
    except ValueError:
        # Manejar errores de conversión de fecha
        print("Error: Formato de fecha incorrecto.")
    
    return date

In [321]:
def ft_solution(result, name):
    result = result[list(result.keys())[0]]
    
    for key, value in result.items():
        if key == "nombre_cliente":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "dni_cliente":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "calle_cliente":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "cp_cliente":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "población_cliente":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "provincia_cliente":
            try:
                result[key] = value[0]
            except:
                pass

        if key == "cif_comercializadora":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "nombre_comercializadora":
            try:
                result[key] = value[0].replace(".", "")
            except:
                pass
        if key == "dirección_comercializadora":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "cp_comercializadora":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "localidad_comercializadora":
            try:
                result[key] = value[0]
            except:
                pass
        if key == "provincia_comercializadora":
            try:
                result[key] = value[0]
            except:
                pass

        if key == "número_factura":
            try:
                result[key] = value
            except:
                pass
        if key == "inicio_periodo":
            try:
                result[key] = ft_date_convert(value)
            except:
                pass
        if key == "fin_periodo":
            try:
                result[key] = ft_date_convert(value)
            except:
                pass
        if key == "fecha_cargo":
            try:
                result[key] = ft_date_convert(value)
            except:
                pass
        if key == "importe_factura":
            try:
                result[key] = value
            except:
                pass
        if key == "consumo_periodo":
            try:
                result[key] = int(value.replace(",", ""))
            except:
                pass
        if key == "potencia_contratada":
            try:
                result[key] = value
            except:
                pass

        with open(f"{name}.json", "w") as archivo_json:
            json.dump(result, archivo_json)
    
    return result


In [312]:
path = "./training/factura_124.pdf"
name = "Factura_124"
factura = ft_readinvoice(path, name)
factura_limpia = ft_normalize(name)
result = ft_font_code(factura_limpia, name)
new_result = ft_solution(result)

Error: Formato de fecha incorrecto.


In [313]:
new_result

{'nombre_cliente': 'carmela dario gamboa',
 'dni_cliente': '24338786w',
 'calle_cliente': 'callede la virgen de africa',
 'cp_cliente': '09640',
 'población_cliente': 'hortiguela',
 'cif_comercializadora': 'b66277138',
 'nombre_comercializadora': '   unic global logistics, sl ',
 'dirección_comercializadora': '. c/ pompeu fabra, 13 2o despacho 10',
 'cp_comercializadora': '08242',
 'localidad_comercializadora': 'manresa',
 'número_factura': 'cj4228927656',
 'inicio_periodo': '07.11.2006',
 'fin_periodo': '09.09.2006',
 'fecha_cargo': '14 de September de 2006',
 'provincia_cliente': 'burgos',
 'importe_factura': '88,48',
 'consumo_periodo': 182,
 'potencia_contratada': '5,41',
 'provincia_comercializadora': 'Barcelona'}

# **Comprobaciones**

In [232]:
diccionario = {}
for key, value in data_train.items():
    for apartado, dic in value.items():
        if apartado == key:
            if (len(dic) != 12):
                diccionario[key] = list(["nombre_cliente",
                            "dni_cliente",
                            "calle_cliente",
                            "cp_cliente",
                            "población_cliente",
                            "cif_comercializadora",
                            "nombre_comercializadora",
                            "dirección_comercializadora",
                            "cp_comercializadora",
                            "localidad_comercializadora",
                            "provincia_comercializadora",
                            "provincia_cliente", 
                            "número_factura",
                            "inicio_periodo",
                            "fin_periodo",
                            "importe_factura",
                            "fecha_cargo",
                            "consumo_periodo",
                            "potencia_contratada"] - dic.keys())
    

In [234]:
provincia_comercializadora = 0
nombre_comercializadora = 0
cp_comercializadora = 0
localidad_comercializadora = 0
dirección_comercializadora = 0
nombre_cliente = 0
dni_cliente = 0
calle_cliente = 0
cp_cliente = 0
población_cliente = 0
cif_comercializadora = 0
provincia_cliente = 0
número_factura = 0
inicio_periodo = 0
fin_periodo = 0
importe_factura = 0
fecha_cargo = 0
consumo_periodo = 0
potencia_contratada = 0

for i, n in diccionario.items():
    if 'nombre_cliente' in diccionario[i]:
        nombre_cliente += 1
    if 'dni_cliente' in diccionario[i]:
        dni_cliente += 1
    if 'calle_cliente' in diccionario[i]:
        calle_cliente += 1
    if 'cp_cliente' in diccionario[i]:
        cp_cliente += 1
    if 'población_cliente' in diccionario[i]:
        población_cliente += 1
    if 'provincia_cliente' in diccionario[i]:
        provincia_cliente += 1
    
    if 'nombre_comercializadora' in diccionario[i]:
        nombre_comercializadora += 1
    if 'cif_comercializadora' in diccionario[i]:
        cif_comercializadora += 1
    if 'dirección_comercializadora' in diccionario[i]:
        dirección_comercializadora += 1
    if 'cp_comercializadora' in diccionario[i]:
        cp_comercializadora += 1
    if 'localidad_comercializadora' in diccionario[i]:
        localidad_comercializadora += 1
    if 'provincia_comercializadora' in diccionario[i]:
        provincia_comercializadora += 1
    
    if 'número_factura' in diccionario[i]:
        número_factura += 1
    if 'inicio_periodo' in diccionario[i]:
        inicio_periodo += 1
    if 'fin_periodo' in diccionario[i]:
        fin_periodo += 1
    if 'importe_factura' in diccionario[i]:
        importe_factura += 1
    if 'fecha_cargo' in diccionario[i]:
        fecha_cargo += 1
    if 'consumo_periodo' in diccionario[i]:
        consumo_periodo += 1
    if 'potencia_contratada' in diccionario[i]:
        potencia_contratada += 1
    
print("nombre_cliente", nombre_cliente)
print("dni_cliente", dni_cliente)
print("calle_cliente", calle_cliente)
print("cp_cliente", cp_cliente)
print("población_cliente", población_cliente)
print("provincia_cliente", provincia_cliente)

print("nombre_comercializadora", nombre_comercializadora)
print("cif_comercializadora", cif_comercializadora)
print("dirección_comercializadora", dirección_comercializadora)
print("cp_comercializadora", cp_comercializadora)
print("localidad_comercializadora", localidad_comercializadora)
print("provincia_comercializadora", provincia_comercializadora)

print("número_factura", número_factura)
print("inicio_periodo", inicio_periodo)
print("fin_periodo", fin_periodo)
print("importe_factura", importe_factura)
print("fecha_cargo", fecha_cargo)
print("consumo_periodo", consumo_periodo)
print("potencia_contratada", potencia_contratada)

nombre_cliente 0
dni_cliente 0
calle_cliente 0
cp_cliente 0
población_cliente 0
provincia_cliente 0
nombre_comercializadora 36
cif_comercializadora 1
dirección_comercializadora 40
cp_comercializadora 48
localidad_comercializadora 46
provincia_comercializadora 42
número_factura 0
inicio_periodo 0
fin_periodo 0
importe_factura 0
fecha_cargo 0
consumo_periodo 0
potencia_contratada 2


------------------------------------------------------------------------------------------------------------------

# **Código final para una factura**

In [322]:
path = "./training/factura_50.pdf"
name = "Factura_50"
factura = ft_readinvoice(path, name)
factura_limpia = ft_normalize(name)
result = ft_font_code(factura_limpia, name)
new_result = ft_solution(result, name)

Factura_50
Factura_50.pkl
Factura_50


In [315]:
new_result

{'nombre_cliente': 'atila nieto olmeda',
 'dni_cliente': '80181585c',
 'calle_cliente': 'plaza de la moreria',
 'cp_cliente': '24723',
 'población_cliente': 'lucillo',
 'cif_comercializadora': 'b93543908',
 'nombre_comercializadora': 'gerenta energia, slu',
 'dirección_comercializadora': 'c/ poeta salvador rueda, no 25, local 2',
 'cp_comercializadora': '29640',
 'localidad_comercializadora': 'malaga',
 'provincia_comercializadora': 'malaga',
 'número_factura': 'yl2724152930',
 'inicio_periodo': '07.26.1996',
 'fin_periodo': '09.24.1996',
 'fecha_cargo': '09.29.1996',
 'consumo_periodo': 774,
 'potencia_contratada': '2,930',
 'importe_factura': '189,49',
 'provincia_cliente': 'León'}